# Generator V1

Ferramenta que pega no conteudo das noticias (em formato json) e gera posts.
Baseia-se nas seguintes regras:

        1) A avaliaçao do deve ficar no inicio do post em letras garrafais.
        2) Em seguida deve ser colocada a alegaçao. Devem ser destacadas as keywords e apresentadas em letras garrafais.
        3) Deve depois aparecer hashtags e emojis associados;
        4) Os elemntos do tweet seguem sempre a mesma ordem.



In [1]:
import os
import re
import json
import pandas as pd
import random
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
from keybert import KeyBERT
import emoji #https://carpedm20.github.io/emoji/

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/opt/homebrew/anaconda3/envs/py3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Extrair avaliçoes communs no snopes
evaluations = []
for file in os.listdir("../../extractors/Snopes/extractions"):
        with open("../../extractors/Snopes/extractions/" + file) as f:
            data = json.load(f)
            f.close()
        if data['evaluation'] not in evaluations:
            evaluations.append(data['evaluation'])

### Gerador

In [2]:
#pip install import-ipynb
import hashtags_generetor
import emojis
import verify_post


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mariabarbosa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
def generate(data):
    
    text = re.sub(r'[^\w\s]', '', data['allegation']) 
    words_allegation = [word.lower() for word in text.strip('\n').split() ]
    
    
    ## Definir o modelo
    kw_model = KeyBERT()
    words_bert = [word for word, value in kw_model.extract_keywords(docs=text, keyphrase_ngram_range=(1,1))]
                                        
    allegation =  []
    
    for word in text.strip('\n').split():
        if word.lower() in words_bert:
            allegation.append(word.upper())
        else:
            allegation.append(word)
    
    allegation = ' '.join(allegation)


    #emoji.emojize(':thumbs_up:')
    post = emoji.emojize(':red_exclamation_mark:') + data['evaluation'].upper() + emoji.emojize(':red_exclamation_mark:') + ' '+ allegation 
    #+ choice(dic[data['evaluation']])
    return post

In [3]:
def main():
    posts = {}
    ##### Read the json files with de news
    for file in os.listdir("../../extractors/Snopes/extractions"):
        with open("../../extractors/Snopes/extractions/" + file) as f:
            data = json.load(f)
            f.close()
            
        x = generate(data)
        
        '''
        Identificar as hashtgs a adicionar.

        1) Identificar keywords do post;
        2) Verificar a similaridade dessas keywords;
        3) 
        '''
       
        hashtags = hashtags_generetor.generateHashtags(' '.join(data['postText']).split('\n'))
        hashtags = [' #'+h for h in hashtags]
        post = x + ' '.join (hashtags) + emojis.emojis_selector(data['evaluation'])
        
        seed = random.randint(0,5)
        
        while (verify_post.verify_length(post) and seed > 0):
                seed = seed - 1
                post = post + emojis.emojis_selector(data['evaluation'])
        print(post)


        df = pd.read_csv('snopes_posts_v1.2.csv')

        row = {'id': data['id'], 'post': post}

        # Adicionar a nova linha ao dataset
        df = df.append(row,  ignore_index=True)

        #Retirar colunaas que estão a mais
        df = df[['id','post' ]]


        df.to_csv('snopes_posts_v1.2.csv')
            

        f.close()
        
        

        
        

In [ ]:
def main():
    posts = {}
    ##### Read the json files with de news
    for file in os.listdir("../../extractors/Snopes/extractions"):
        with open("../../extractors/Snopes/extractions/" + file) as f:
            data = json.load(f)
            f.close()
            
        x = generate(data)
        
        '''
        Identificar as hashtgs a adicionar.

        1) Identificar keywords do post;
        2) Verificar a similaridade dessas keywords;
        3) 
        '''
       
        hashtags = hashtags_generetor.generateHashtags(' '.join(data['postText']).split('\n'))
        hashtags = [' #'+h for h in hashtags]
        post = x + ' '.join (hashtags) + emojis.emojis_selector(data['evaluation'])
        
        seed = random.randint(0,5)
        
        while (verify_post.verify_length(post) and seed > 0):
                seed = seed - 1
                post = post + emojis.emojis_selector(data['evaluation'])
        print(post)


        df = pd.read_csv('snopes_posts_v1.2.csv')

        row = {'id': data['id'], 'post': post}

        # Adicionar a nova linha ao dataset
        df = df.append(row,  ignore_index=True)

        #Retirar colunaas que estão a mais
        df = df[['id','post' ]]


        df.to_csv('snopes_posts_v1.2.csv')
            

        f.close()
        
        

        
        

In [8]:
main()

❗FALSE❗ VALENTINES DAY was INVENTED by GREETING CARD companies #holiday😪🔫🔫😫🔫💀
❗MISCAPTIONED❗ A VIDEO shows TRUCKERS from South CAROLINA on their way to join a FREEDOM CONVOY in Canada #truck😩😔
❗FALSE❗ An online ADVERTISEMENT revealed an unusual or unique tip or TRICK for CARS with AUTOMATIC HEADLIGHTS #automatic😒😒😴😒
❗FALSE❗ RUSSIA BOMBED a Bidenowned VILLA and several US BIOLABS in UKRAINE #real😞😔█🔫💀
❗TRUE❗ In September 2021 an initial news release announcing US Rep LAUREN BOEBERTS IMPEACHMENT articles again President Joe BIDEN and Vice President Kamala Harris included the misspelled slogan IMEACH BIDEN #president😻
❗FALSE❗ A StanfordNIH study concluded that FACE MASKS worn to PREVENT COVID19 TRANSMISSION do not work #scientific  #medical😪😫
❗CORRECT-ATTRIBUTION❗ In 1527 MARTIN LUTHER wrote of the BLACK Death I shall ask God to mercifully PROTECT us and I shall not avoid person or place but will go freely #plague  #death😒😞😭█
❗FALSE❗ In late August 2021 Vice President Kamala Harris mistak